In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:

# Đọc dữ liệu
file_path = '/content/drive/MyDrive/DoanPhuongNam_HuynhHuuThinh/Vietnamese News Text Classification/train_data (1).csv'
df = pd.read_csv(file_path, encoding="utf-16")

# Mã hóa nhãn
label_encoder = LabelEncoder()
df['label_encoded'] = label_encoder.fit_transform(df['label'])
# Tokenization
max_words = 10000  # Giới hạn số từ
max_len = 500  # Độ dài tối đa của văn bản
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')
# Xử lý mất cân bằng dữ liệu bằng SMOTE
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, df['label_encoded'], test_size=0.2, random_state=42)
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Lưu dữ liệu sau xử lý
save_path = '/content/drive/MyDrive/DoanPhuongNam_HuynhHuuThinh/Vietnamese News Text Classification'
os.makedirs(save_path, exist_ok=True)

np.save(save_path + "/X_train_resampled.npy", X_train_resampled)
np.save(save_path + "/y_train_resampled.npy", y_train_resampled)
np.save(save_path + "/X_test.npy", X_test)
np.save(save_path + "/y_test.npy", y_test)


In [8]:
import tensorflow as tf
X_train_resampled = np.load(save_path+"/X_train_resampled.npy")
y_train_resampled = np.load(save_path+"/y_train_resampled.npy")
X_test = np.load(save_path+"/X_test.npy")
y_test = np.load(save_path+"/y_test.npy")

# Xây dựng mô hình LSTM
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=max_words, output_dim=128, input_length=max_len),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
])

# Compile mô hình
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Huấn luyện mô hình
model.fit(X_train_resampled, y_train_resampled, epochs=10, batch_size=64, validation_data=(X_test, y_test))

# Đánh giá mô hình
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%')


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
661/661 ━━━━━━━━━━━━━━━━━━━━ 35s 43ms/step - accuracy: 0.1457 - loss: 2.2399 - val_accuracy: 0.1007 - val_loss: 2.2655
Epoch 2/10
661/661 ━━━━━━━━━━━━━━━━━━━━ 28s 42ms/step - accuracy: 0.1523 - loss: 2.2385 - val_accuracy: 0.2510 - val_loss: 1.9025
Epoch 3/10
661/661 ━━━━━━━━━━━━━━━━━━━━ 42s 44ms/step - accuracy: 0.2363 - loss: 1.9961 - val_accuracy: 0.3709 - val_loss: 1.5832
Epoch 4/10
661/661 ━━━━━━━━━━━━━━━━━━━━ 41s 44ms/step - accuracy: 0.3137 - loss: 1.7490 - val_accuracy: 0.5124 - val_loss: 1.3023
Epoch 5/10
661/661 ━━━━━━━━━━━━━━━━━━━━ 41s 44ms/step - accuracy: 0.4237 - loss: 1.4986 - val_accuracy: 0.6265 - val_loss: 1.0429
Epoch 6/10
661/661 ━━━━━━━━━━━━━━━━━━━━ 41s 44ms/step - accuracy: 0.5149 - loss: 1.3251 - val_accuracy: 0.7679 - val_loss: 0.7631
Epoch 7/10
661/661 ━━━━━━━━━━━━━━━━━━━━ 41s 44ms/step - accuracy: 0.6109 - loss: 1.1108 - val_accuracy: 0.8334 - val_loss: 0.5691
Epoch 8/10
661/661 ━━━━━━━━━━━━━━━━━━━━ 30s 45ms/step - accuracy: 0.6406 - loss: 1.0159 - 

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

def test_model(model, tokenizer, label_encoder, text, max_len=100):
    # Tokenize the input text
    sequences = tokenizer.texts_to_sequences([text])
    # Pad the sequence to ensure it has the same length as the input length used during training
    padded_sequence = pad_sequences(sequences, maxlen=max_len)
    # Get the model's prediction
    prediction = model.predict(padded_sequence)
    # Decode the prediction
    predicted_class = np.argmax(prediction, axis=1)
    predicted_label = label_encoder.inverse_transform(predicted_class)
    return predicted_label[0]
# Example usage:
text_to_test = " Bay qua hang động hẹp Một nhóm phi công Nga sẽ lái máy bay chiến đấu Sukhoi Su-30 và Su- 27 qua một hang động hẹp ở tỉnh Hồ Nam, Trung Quốc trong một màn biểu diễn nguy hiểm mà người muốn xem phải trả 840 USD.Chuyến bay qua động Thiên Môn với kích thước 57 mét ở điểm rộng nhất và 28 mét ở điểm hẹp nhất sẽ diễn ra vào hai ngày 17 và 18-3 tới. Trong ảnh là loại máy bay chiến đấu sẽ được sử dụng cho các pha mạo hiểm nói trên"
predicted_class = test_model(model, tokenizer, label_encoder, text_to_test)
print(f"Predicted class: {predicted_class}")
